In [ ]:
import pydeck as pdk
import geopandas as gpd
import pandas as pd
import os

In [ ]:
# 데이터는 서울 행정동 데이터이다
geo_data = './older_seoul.geojson'

In [ ]:
df = gpd.read_file(geo_data)
df.head()

In [ ]:
# pydeck은 geometry 컬럼의 shapely.geometry.multipolygon 타입을 읽지 못하여 아래와 같이 변경하는 작업을 해줌
# multipolygon을 polygon들로 변경 후 진행하여야 함

df['polygons'] = df['geometry'].apply(lambda x : list(x.geoms))

def polygon_to_coordinates(x):
    lon, lat = x[0].exterior.xy
    return [[x, y] for x, y in zip(lon, lat)]

df['coordinates'] = df['polygons'].apply(polygon_to_coordinates)
del df['geometry']
del df['polygons']

In [ ]:
df.head()

In [ ]:
df['정규화인구'] = df['인구'] / df['인구'].max()

In [ ]:
# pydeck은 pandas.dataframe에서만 동작해서 변경함
df = pd.DataFrame(df)

In [ ]:
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    df, # 시각화에 쓰일 데이터프레임
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 255*정규화인구, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

# Set the viewport location
center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

# Render
r = pdk.Deck(layers=[layer], 
             initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'PolygonLayer', # 사용할 Layer 타입
    df, # 시각화에 쓰일 데이터프레임
    extruded = True,
    get_elevation = '인구',
    elevation_scale = 0.05,
    get_polygon='coordinates', # geometry 정보를 담고있는 컬럼 이름
    get_fill_color='[0, 255*정규화인구, 0]', # 각 데이터 별 rgb 또는 rgba 값 (0~255)
    pickable=True, # 지도와 interactive 한 동작 on
    auto_highlight=True # 마우스 오버(hover) 시 박스 출력
)

view_state.bearing=15
view_state.pitch=45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
coordinates_list = []
for i in range(len(df['coordinates'])):
    for j in range(len(df['coordinates'][i])):
        coordinates_list.append(df['coordinates'][i][j])

In [ ]:
coordinates_df = pd.DataFrame(coordinates_list, columns=['lng','lat'])

In [ ]:
coordinates_df.head()

In [ ]:
layer = pdk.Layer(
    'HexagonLayer',
    coordinates_df,
    get_position='[lng, lat]',
    auto_highlight=True,
    elevation_scale=50,
    pickable=True,
    elevation_range=[0, 3000],
    extruded=True,                 
    coverage=1)
# center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=126.986,
    latitude=37.565,
    zoom=6,
    min_zoom=5,
    max_zoom=15,
    pitch=40.5,
    bearing=-27.36)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
import requests

url = 'http://openAPI.seoul.go.kr:8088/52454e74527a686f3939666a665574/json/SearchPublicToiletPOIService/1/3/'

data1 = requests.get(url).json() 

In [ ]:
api_cols = pd.DataFrame(data1['SearchPublicToiletPOIService']['row']).columns

In [ ]:
aa = [x*1000 for x in range(1,6)]

PT_df = pd.DataFrame(columns = api_cols)

for i in range(len(aa)):
    start_num = aa[i]-999
    end_num = aa[i]

    url = f'http://openAPI.seoul.go.kr:8088/52454e74527a686f3939666a665574/json/SearchPublicToiletPOIService/{start_num}/{end_num}/'

    url2json = requests.get(url).json()
    data = pd.DataFrame(url2json['SearchPublicToiletPOIService']['row'])

    PT_df = pd.concat([PT_df, data], axis=0)

In [ ]:
PT_df.reset_index(drop=True, inplace = True)

In [ ]:
layer = pdk.Layer(
    'ScatterplotLayer',
    PT_df,
    get_position='[X_WGS84, Y_WGS84]',
    get_radius=50,
    get_fill_color='[255, 255, 255]',
    pickable=True,
    auto_highlight=True
)

center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'HeatmapLayer',
    PT_df,
    get_position='[X_WGS84, Y_WGS84]',
    get_radius=50,
    get_fill_color='[255, 255, 255]',
    pickable=True,
    auto_highlight=True
)

center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer = pdk.Layer(
    'CPUGridLayer',
    PT_df,
    get_position='[X_WGS84, Y_WGS84]',
    get_radius=50,
    get_fill_color='[255, 255, 255]',
    pickable=True,
    auto_highlight=True
)

center = [126.986, 37.565]
view_state = pdk.ViewState(
    longitude=center[0],
    latitude=center[1],
    zoom=10)

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer.extruded = True
layer.elevation_scale = 3 # default 1

view_state.bearing = -15
view_state.pitch = 45

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()

In [ ]:
layer.type = 'HexagonLayer'

r = pdk.Deck(layers=[layer], initial_view_state=view_state)
r.to_html()